# 作业：新华社新闻抄袭自动判别
描述：
- 报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

- 给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。
+ 1. 构建一个机器学习模型，判断这个文章是不是新华社的
+ 2. 当这个模型的acc 大于 0.8778， recall， precision，f1等值都较高的时候
+ 3. 用该模型 判断一篇文章是否是新华社的文章，如果判断出来是新华社的，但是，它的source并不是新华社的，那么，我们就说，这个文章是抄袭的新华社的文章

# Step1: 数据分析
请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [1]:
fname = 'C:\\Users\DELL\\Desktop\\NLP\\course1\\export_sql_1558435\\sqlResult_1558435.csv'  # 需要修改数据集的绝对路径

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import jieba
import scipy
import sklearn
import time, timeit
import re

In [201]:
contents = pd.read_csv(fname, encoding='gb18030')

In [202]:
contents.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [203]:
xinhua_news = contents[contents['source'] == '新华社']

In [204]:
len(xinhua_news)

78661

In [205]:
len(xinhua_news) / len(contents)

0.8778051801676133

# Step2: 数据预处理
将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 

你可能要使用到pandas的dataframe操作。https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [206]:
type(contents) # 已经是DataFrame类型了

pandas.core.frame.DataFrame

In [207]:
def cut(string):
    return ''.join(jieba.cut(string))

In [269]:
import re

def token(string):
    str_list = re.findall(r'[\d|\w]+', string) # 只要数字和文字
    pre_str = ''.join(str_list)
    if len(pre_str) > 0:
        return ''.join(jieba.cut(re.sub('n','',pre_str)))
    else:
        return ''

In [270]:
contents = contents.fillna('')  # 填充空值

+ 8万条新闻处理太慢了，我等不了这么久，所以抽取十分之一的新闻来训练和测试

In [271]:
contents = contents.sample(n=10000)  # 序列a中随机抽取n个元素，并将n个元素生以list形式返回

In [272]:
len(contents)

10000

In [273]:
contents['content'] = contents['content'].apply(token)

In [274]:
contents['title'] = contents['title'].apply(token)

In [275]:
contents.columns  

Index(['id', 'author', 'source', 'content', 'feature', 'title', 'url',
       'is_xinhua'],
      dtype='object')

- 再加一列来做标签，当y=1时标记为新华社的文章，否则不是

In [276]:
contents['is_xinhua'] = np.where(contents['source'].str.contains('新华'), 1, 0)

In [277]:
contents.head()

,id,author,source,content,feature,title,url,is_xinhua
10634,78979,,河南日报第03版,本报讯6月10日雨天的早晨天亮得晚一些但南阳市卧龙区蒲山镇师杨庄村贫困户袁玉敏已经早早起床伺...,"{""type"":""要闻"",""site"":""河南日报"",""commentNum"":""0"",""j...",车间进农户脱贫有出路,http://newpaper.dahe.cn/hnrb/html/2017-06/14/c...,0
74690,14922,王方（湖北）,新华社,新华社照片武汉2017年5月25日湖北消防部队开展抗洪抢险救援演练5月25日消防官兵驾驶水陆...,"{""type"":""军事教育与训练"",""site"":""新华社"",""url"":""http://h...",社会1湖北消防部队开展抗洪抢险救援演练,http://home.xinhua-news.com/gdsdetailxhsnew/21...,1
6672,82945,,经济参考报,工业和信息化部部长苗圩19日在参观青海省西宁市南川工业园区时表示未来将进一步推进新能源汽车顶...,"{""type"":""新闻"",""site"":""中证网"",""commentNum"":""0"",""jo...",工信部将推进新能源车顶层设计多领域政策将制定,http://www.cs.com.cn/xwzx/201706/t20170620_533...,0
69358,20254,罗欢欢,新华社,新华社照片科隆德国2017年5月21日体育4冰球世锦赛加拿大晋级决赛5月20日加拿大队球员沙...,"{""type"":""体育"",""site"":""新华社"",""url"":""http://home.x...",体育4冰球世锦赛加拿大晋级决赛,http://home.xinhua-news.com/gdsdetailxhsnew/21...,1
33812,55800,林宏,新华社,新华社照片农安吉林2017年4月19日吉林春耕生产忙4月19日吉林省农安县陈家窝堡村农民在地...,"{""type"":""行业经济"",""site"":""新华社"",""url"":""http://home...",经济2吉林春耕生产忙,http://home.xinhua-news.com/gdsdetailxhsnew/19...,1


In [278]:
print(contents.iloc[0]['id'])  # 访问其中的元素

78979


In [279]:
xinhua_news = contents[contents['source'] == '新华社']

In [280]:
len(xinhua_news)

8745

In [281]:
len(xinhua_news) / len(contents)  # 随机提取8000条新闻和全部80000条新闻的比例是一样的

0.8745

# Step3: 使用tfidf进行文本向量化
参考 https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 对

对文本进行向量化

In [282]:
X_title = contents['title'].values  # 提取title的内容,已经cut()完了

In [284]:
# for title in X_title:
#     if title==None:
#         print(title)

In [285]:
X_content = contents['content'].values # 提取contents的内容

In [286]:
y = contents['is_xinhua'].values  # 标记值

In [287]:
print(len(X_title),len(X_content),len(y))

10000 10000 10000


In [288]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [289]:
vectorizer = TfidfVectorizer(max_features=600)  # 提取排名前10000个特征

In [290]:
X_title = vectorizer.fit_transform(X_title)

In [291]:
X_content = vectorizer.fit_transform(X_content) 

In [292]:
X_title.shape   # 8000条新闻内容，每一条新闻内容有10000个特征

(10000, 600)

In [293]:
X_content.shape # 就是个矩阵

(10000, 600)

- **把新闻id和向量之间建立一个映射，之后预测不是新华社的新闻可以还原出来**

- 假定向量与原来新闻之间是有序的，是按照顺序进行的向量化

In [294]:
newsID_vec = {} # key为向量(稀疏矩阵的表现形式) value为id,
for i in range(len(contents)):
    newsID_vec[contents.iloc[i]['id']] = X_content[i] 

# Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型
按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. 
- 参考：https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

参照scikit learning的示例，从构建你的第一个KNN机器学习模型。
- 参考：https://scikit-learn.org/stable/modules/neighbors.html#classification
     https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.fit

- 把**标题**划分训练集和测试集

In [295]:
X_t_train, X_t_test, y_t_train, y_t_test = train_test_split(X_title, y, test_size=0.25, random_state=0)  

- 把**新闻内容**划分训练集和测试集

In [296]:
X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(X_content, y, test_size=0.25, random_state=0)

In [297]:
X_t_train.shape

(7500, 600)

In [298]:
X_t_test.shape

(2500, 600)

# 以下过程是对上述划分函数的功能的具体实现

In [83]:
indices = np.arange(len(y)) # 下标从0-7999

In [84]:
np.random.shuffle(indices)  # 把原来数组随机排列一次，打乱顺序

In [85]:
split_test = 0.75  # 这里有不一致的地方

In [86]:
train_indices = indices[:int(len(indices)*split_test)]

In [87]:
len(train_indices)

6000

In [88]:
test_indices = indices[int(len(indices)*split_test):]

In [89]:
len(test_indices)

2000

In [90]:
X_t_train, X_t_test, y_train, y_test = (
    X_title[train_indices], X_title[test_indices], 
    y[train_indices], y[test_indices])

In [91]:
X_t_train

<6000x600 sparse matrix of type '<class 'numpy.float64'>'
	with 22176 stored elements in Compressed Sparse Row format>

In [92]:
X_t_test

<2000x600 sparse matrix of type '<class 'numpy.float64'>'
	with 7378 stored elements in Compressed Sparse Row format>

In [93]:
len(y_train)

6000

In [94]:
len(y_test)

2000

# 先把新闻标题作为训练集，然后进行分类和预测

In [299]:
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, datasets

# Based on Title or Based on Content?
Answer: Just Try it

In [308]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [309]:
def get_performance(clf, X_, y_, sample=1000):  # sample表示取出多少个样例出来
    random_indices = np.random.choice(np.arange(len(y_)), sample) 
    sub_x = X_[random_indices]  # 实际的X值
    sub_y = y_[random_indices]  # 实际的Y值
    y_hat = clf.predict(sub_x.toarray())  # 预测的Y值
    print('percision is: {}'.format(precision_score(sub_y, y_hat)))
    print('recall is: {}'.format(recall_score(sub_y, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(sub_y, y_hat)))
    print('confusion matrix: \n{}'.format(confusion_matrix(sub_y, y_hat, labels=[0, 1])))

In [310]:
# we create an instance of Neighbours Classifier and fit the data.
clf_t = neighbors.KNeighborsClassifier(n_neighbors=6, weights='uniform')  # or weights='distance'
clf_t.fit(X_t_train.toarray(), y_t_train)
get_performance(clf_t, X_=X_t_train, y_=y_t_train, sample=7500)  # 所有测试集就是7500

percision is: 0.8732583065380493
recall is: 0.9986211123027424
roc_auc is: 0.5131851707454103
confusion matrix: 
[[  27  946]
 [   9 6518]]


# Step7: 调整不同的参数，观察变化（主要是调整邻居的个数，此外还有特征个数）

In [311]:
clf_t = neighbors.KNeighborsClassifier(n_neighbors=8, weights='uniform')
clf_t.fit(X_t_train.toarray(), y_t_train)
get_performance(clf_t, X_=X_t_train, y_=y_t_train, sample=7500)

percision is: 0.8664617029808849
recall is: 0.9992292276861415
roc_auc is: 0.506524781661432
confusion matrix: 
[[  14  999]
 [   5 6482]]


In [312]:
clf_t = neighbors.KNeighborsClassifier(n_neighbors=12, weights='uniform')
clf_t.fit(X_t_train.toarray(), y_t_train)
get_performance(clf_t, X_=X_t_train, y_=y_t_train, sample=7500)

percision is: 0.8762845322300814
recall is: 1.0
roc_auc is: 0.503747323340471
confusion matrix: 
[[   7  927]
 [   0 6566]]


In [313]:
clf_t = neighbors.KNeighborsClassifier(n_neighbors=20, weights='uniform')
clf_t.fit(X_t_train.toarray(), y_t_train)
get_performance(clf_t, X_=X_t_train, y_=y_t_train, sample=7500)

percision is: 0.8670666666666667
recall is: 1.0
roc_auc is: 0.5
confusion matrix: 
[[   0  997]
 [   0 6503]]


By definition a confusion matrix $ C  $ is such that $ C_{i, j} $ is equal to the number of observations known to be in group $ i $ but predicted to be in group $ j $.

Thus in binary classification, the count of true negatives is $ C_{0,0} $, false negatives is $ C_{1,0} $ , true positives is $ C_{1,1} $ and false positives is $ C_{0,1} $.

- percision = 465/(465+1) = 0.9978
- recall = 465 / (465+1273) = 0.26

# 把新闻内容作为训练集，然后进行分类和预测,从这一行开始再次运行

In [316]:
clf_c = neighbors.KNeighborsClassifier(n_neighbors=6, weights='uniform')  # or weights='distance'
clf_c.fit(X_c_train.toarray(), y_c_train)  # 基于内容来分类

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')

In [317]:
get_performance(clf_c, X_=X_c_train, y_=y_c_train, sample=7500)  # 训练集就6000条数据

percision is: 0.8787716955941255
recall is: 0.9993926510780443
roc_auc is: 0.5029786012501819
confusion matrix: 
[[   6  908]
 [   4 6582]]


In [318]:
clf_c = neighbors.KNeighborsClassifier(n_neighbors=8, weights='uniform')  # or weights='distance'
clf_c.fit(X_c_train.toarray(), y_c_train)  # 基于内容来分类
get_performance(clf_c, X_=X_c_train, y_=y_c_train, sample=7500)  # 训练集就6000条数据

percision is: 0.8783621035728623
recall is: 0.9977200182398541
roc_auc is: 0.505374667100383
confusion matrix: 
[[  12  909]
 [  15 6564]]


In [319]:
clf_c = neighbors.KNeighborsClassifier(n_neighbors=12, weights='uniform')  # or weights='distance'
clf_c.fit(X_c_train.toarray(), y_c_train)  # 基于内容来分类
get_performance(clf_c, X_=X_c_train, y_=y_c_train, sample=7500)  # 训练集就6000条数据

percision is: 0.8685965848452508
recall is: 1.0
roc_auc is: 0.5020222446916076
confusion matrix: 
[[   4  985]
 [   0 6511]]


In [127]:
clf_c = neighbors.KNeighborsClassifier(n_neighbors=20, weights='uniform')  # or weights='distance'
clf_c.fit(X_c_train.toarray(), y_c_train)  # 基于内容来分类
get_performance(clf_c, X_=X_c_train, y_=y_c_train, sample=7500)  # 训练集就6000条数据

percision is: 0.9872589531680441
recall is: 0.5453680806543656
roc_auc is: 0.747784982453697
confusion matrix: 
[[ 706   37]
 [2390 2867]]


# Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者
- 思路是把所有标记为不是新华社的文章进行预测，如果预测值为新华社的文章则判定为抄袭新华社

In [359]:
fakeNews = contents[contents['source'] != '新华社']  # 这只是语料库中的一部分

In [360]:
len(fakeNews)

1255

In [362]:
fakeNews[0:2]

,id,author,source,content,feature,title,url,is_xinhua
10634,78979,,河南日报第03版,本报讯6月10日雨天的早晨天亮得晚一些但南阳市卧龙区蒲山镇师杨庄村贫困户袁玉敏已经早早起床伺...,"{""type"":""要闻"",""site"":""河南日报"",""commentNum"":""0"",""j...",车间进农户脱贫有出路,http://newpaper.dahe.cn/hnrb/html/2017-06/14/c...,0
6672,82945,,经济参考报,工业和信息化部部长苗圩19日在参观青海省西宁市南川工业园区时表示未来将进一步推进新能源汽车顶...,"{""type"":""新闻"",""site"":""中证网"",""commentNum"":""0"",""jo...",工信部将推进新能源车顶层设计多领域政策将制定,http://www.cs.com.cn/xwzx/201706/t20170620_533...,0


In [363]:
fake_content = fakeNews['content'].values # 提取contents的内容

In [364]:
fake_y = fakeNews['is_xinhua'].values  # 标记值

In [366]:
vectorizer = TfidfVectorizer(max_features=600)  # 提取排名前600个特征

In [367]:
fake_content = vectorizer.fit_transform(fake_content) 

In [368]:
fake_content.shape

(1255, 600)

- 基于新闻内容训练出来的分类器来进行预测

In [369]:
clf_c = neighbors.KNeighborsClassifier(n_neighbors=6, weights='uniform')  # or weights='distance'
clf_c.fit(X_c_train.toarray(), y_c_train)  # 基于内容来分类

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')

In [370]:
fake_y_predict = clf_c.predict(fake_content.toarray())  # 对测试集进行预测，然后与标签进行比较

In [372]:
len(fake_y_predict)

1255

In [373]:
count_fake = 0
for i in range(len(fake_y_predict)):
    if fake_y_predict[i]==1:
        count_fake += 1

In [374]:
count_fake

1246

In [376]:
get_performance(clf_c, X_=fake_content, y_=fake_y, sample=1255)  # 训练集就1250条数据

percision is: 0.012810248198558846
recall is: 0.9411764705882353
roc_auc is: 0.47260762140074125
confusion matrix: 
[[   5 1233]
 [   1   16]]


In [324]:
# y_c_test 是实际的标签

In [325]:
potential_copy_index = [no for no,is_xinhua in enumerate(y_c_test) if is_xinhua==0 and y_test_predict[no]==1]#预测为 1，但是实际为0的文章下标

In [333]:
len(potential_copy_index)

286

In [326]:
# 根据index再获得新闻内容 X_c_train, X_c_test, y_c_train, y_c_test 

In [327]:
potential_copy_vector = [X_c_test[no] for no in potential_copy_index]

In [335]:
len(potential_copy_vector)

286

In [336]:
potential_copy_vector[0]

<1x600 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [339]:
len(newsID_vec)

10000

# 只找到了新闻向量化后的结果，怎么还原到原始的新闻呢?????
建立了一个dict来映射向量与新闻

In [354]:
fake_news_id = []
for vec in potential_copy_vector:  # 256
    for key, value in newsID_vec.items(): # 10000
        if np.all(vec.toarray()==value.toarray())==True and key not in fake_news_id: # 1*600 * 256
            fake_news_id.append(key)

In [355]:
len(fake_news_id)

9001

In [356]:
fake_news_id[0:20]

[78979,
 14922,
 82945,
 20254,
 55800,
 10684,
 43058,
 75257,
 44818,
 30149,
 5566,
 70452,
 60149,
 21412,
 5054,
 62833,
 13636,
 12496,
 24302,
 80646]

In [347]:
contents[contents['id'] == 78979]

,id,author,source,content,feature,title,url,is_xinhua
10634,78979,,河南日报第03版,本报讯6月10日雨天的早晨天亮得晚一些但南阳市卧龙区蒲山镇师杨庄村贫困户袁玉敏已经早早起床伺...,"{""type"":""要闻"",""site"":""河南日报"",""commentNum"":""0"",""j...",车间进农户脱贫有出路,http://newpaper.dahe.cn/hnrb/html/2017-06/14/c...,0


In [348]:
contents.columns

Index(['id', 'author', 'source', 'content', 'feature', 'title', 'url',
       'is_xinhua'],
      dtype='object')

In [349]:
fake_news = pd.DataFrame(columns=['id', 'author', 'source', 'content', 'feature', 'title', 'url', 'is_xinhua'])
for id in fake_news_id:
    fake_news = fake_news.append(contents[contents['id'] == id])

In [350]:
fake_news.head()

,id,author,source,content,feature,title,url,is_xinhua
10634,78979,,河南日报第03版,本报讯6月10日雨天的早晨天亮得晚一些但南阳市卧龙区蒲山镇师杨庄村贫困户袁玉敏已经早早起床伺...,"{""type"":""要闻"",""site"":""河南日报"",""commentNum"":""0"",""j...",车间进农户脱贫有出路,http://newpaper.dahe.cn/hnrb/html/2017-06/14/c...,0
10634,78979,,河南日报第03版,本报讯6月10日雨天的早晨天亮得晚一些但南阳市卧龙区蒲山镇师杨庄村贫困户袁玉敏已经早早起床伺...,"{""type"":""要闻"",""site"":""河南日报"",""commentNum"":""0"",""j...",车间进农户脱贫有出路,http://newpaper.dahe.cn/hnrb/html/2017-06/14/c...,0
10634,78979,,河南日报第03版,本报讯6月10日雨天的早晨天亮得晚一些但南阳市卧龙区蒲山镇师杨庄村贫困户袁玉敏已经早早起床伺...,"{""type"":""要闻"",""site"":""河南日报"",""commentNum"":""0"",""j...",车间进农户脱贫有出路,http://newpaper.dahe.cn/hnrb/html/2017-06/14/c...,0
10634,78979,,河南日报第03版,本报讯6月10日雨天的早晨天亮得晚一些但南阳市卧龙区蒲山镇师杨庄村贫困户袁玉敏已经早早起床伺...,"{""type"":""要闻"",""site"":""河南日报"",""commentNum"":""0"",""j...",车间进农户脱贫有出路,http://newpaper.dahe.cn/hnrb/html/2017-06/14/c...,0
10634,78979,,河南日报第03版,本报讯6月10日雨天的早晨天亮得晚一些但南阳市卧龙区蒲山镇师杨庄村贫困户袁玉敏已经早早起床伺...,"{""type"":""要闻"",""site"":""河南日报"",""commentNum"":""0"",""j...",车间进农户脱贫有出路,http://newpaper.dahe.cn/hnrb/html/2017-06/14/c...,0


# Step10： 总结该过程，什么是数据思维？什么是机器学习思维？
- 总结：step1-step9，步骤很清晰，每一步所做的工作很明确
- 数据思维：通过统计分析，推理等，发现原始数据之间的规律，然后利用规律去处理其他数据
- 机器学习思维：建立模型，通过大量的原始数据进行训练，然后利用模型对新数据进行预测

# --------------------------------------Assignments-08----------------------------------------------
- 1. Using the new data to test different models 

  • KNN, **Logistic Regression, SVM, Bayes, Random Forest, XGBoost** 

  • To Classify if a new a by Xinhua.
  
  
- 2. Summary the advantages and disadvantages about Linear Regression, Logistic Regression, K-Nearest Neighbors, Support Vector Machine, Bayes, etc. 

# Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响

### 1:逻辑回归 

In [378]:
from sklearn.linear_model import LogisticRegression
clfLR = LogisticRegression(solver='saga').fit(X_c_train.toarray(), y_c_train)
get_performance(clfLR, X_=X_t_train, y_=y_train, sample=7500)

percision is: 0.8797704830531091
recall is: 0.9990907713289893
roc_auc is: 0.49954538566449463
confusion matrix: 
[[   0  901]
 [   6 6593]]


### 2:支持向量机

In [381]:
from sklearn.svm import LinearSVC
clfSVM = LinearSVC(dual=False).fit(X_c_train.toarray(), y_c_train)
get_performance(clfSVM, X_=X_t_train, y_=y_train, sample=7500)

percision is: 0.8713637576728049
recall is: 0.9990820073439413
roc_auc is: 0.49954100367197063
confusion matrix: 
[[   0  964]
 [   6 6530]]


### 3:高斯贝叶斯分类器

In [382]:
from sklearn.naive_bayes import GaussianNB  # GaussianNB就是先验为高斯分布的朴素贝叶斯
clfNB = GaussianNB()  # 高斯贝叶斯分类器,还有NultinomialNB和BernoulliNB这两个，用法是一样的
clfNB.fit(X_c_train.toarray(), y_c_train)
get_performance(clfNB, X_=X_t_train, y_=y_train, sample=7500)

percision is: 0.8815533980582524
recall is: 0.06891317547055252
roc_auc is: 0.5010136052791359
confusion matrix: 
[[ 851   61]
 [6134  454]]


### 4:随机森林

In [386]:
from sklearn.ensemble import RandomForestClassifier
clfRFC = RandomForestClassifier(n_estimators=100, random_state=0).fit(X_c_train.toarray(), y_c_train)
get_performance(clfRFC, X_=X_t_train, y_=y_train, sample=7500)

percision is: 0.8779869176344948
recall is: 0.9986334649255998
roc_auc is: 0.4993167324627999
confusion matrix: 
[[   0  914]
 [   9 6577]]


### 5:XGBoost

### 6:决策树模型

In [384]:
from sklearn.tree import DecisionTreeClassifier
clfDTC = DecisionTreeClassifier().fit(X_c_train.toarray(), y_c_train)
get_performance(clfDTC, X_=X_t_train, y_=y_train, sample=7500)

percision is: 0.8776354416866827
recall is: 0.9990885614461492
roc_auc is: 0.4995442807230746
confusion matrix: 
[[   0  917]
 [   6 6577]]


# 总结优缺点
https://zhuanlan.zhihu.com/p/46831267